In [3]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
csv_file = pd.read_csv(f"{location}/ERC_correlation.csv")

csv_file.index = csv_file.columns
string_to_remove = "_nostop.fas.aligned.fas.best"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

string_to_remove = ".fas.ClipKIT"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


matrix_long = csv_file.reset_index().melt(
    id_vars="index",         # Keeps row names as a column
    var_name="Gene1",       # New name for columns
    value_name="Gene2"       # New name for matrix values
)

# Rename 'index' to 'Row' for clarity
diapause = matrix_long.rename(columns={"index": "Row"})

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"
# location = "/mnt/c/Users/Saurav Baral/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)

diapause_rn["Correlation"].fillna(1, inplace=True)

# print(diapause_rn)

genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.1]
# diapause_rn["Correlation"] += 1
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK"]
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation

gene_dict = gene_dictionary
print("done")

done


In [16]:
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation
            
            if not(row.Correlation <= 1.0 and row.Correlation >= -1.0):
                print(row.Correlation <= 1.0)
                print(row.Annotation_gene1 , row.Annotation_gene2, row.Correlation )
                assert False
# gene_dict = gene_dictionary
print("done")

done


In [6]:
result = diapause_rn.loc[diapause_rn["Correlation"].isna(), ["Gene1","Gene2"]]

In [8]:
diapause_rn

,Gene1,Gene2,Correlation,Annotation_gene1,Annotation_gene2
0,7556at7088,7556at7088,1.000000,npr-9,npr-9
1,14105at7088,7556at7088,0.185247,ACOT13,npr-9
2,7117at7088,7556at7088,0.194968,NPEPL1,npr-9
3,8175at7088,7556at7088,0.138848,NR2C2,npr-9
4,12496at7088,7556at7088,0.032970,PIGF,npr-9
...,...,...,...,...,...
20894036,7675at7088,4241at7088,0.107122,HDAC8,SF3B2
20894037,360at7088,4241at7088,0.183250,KCNT2,SF3B2
20894038,13468at7088,4241at7088,-0.037958,13468at7088,SF3B2
20894039,3921at7088,4241at7088,0.345078,TUBG2,SF3B2


In [30]:
# print(gene_dict)

In [18]:
import networkx as nx
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform

import random
# gene_dict = dict(random.sample(list(gene_dictionary.items()), 1000))
gene_dict = gene_dictionary

# Create a graph
# G = nx.Graph()

# Add edges with similarity weights
# for gene, neighbors in gene_dict.items():
#     for neighbor, similarity in neighbors.items():
#         G.add_edge(gene, neighbor, weight=similarity)

# List of genes
genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))

print(similarity_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
# Fill the similarity matrix and ensure symmetry
# import random
# gene_dict = dict(random.sample(list(gene_dictionary.items()), 20))


for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = round(gene_dict[gene1][gene2],2)
            similarity_matrix[j, i] = round(gene_dict[gene1][gene2],2)  # Ensure symmetry
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)
            similarity_matrix[j, i] = 0


min_similarity = np.min(similarity_matrix)
# similarity_matrix = np.nan_to_num(similarity_matrix, nan=0.0)
print(min_similarity)
# if min_similarity < 0:
#     similarity_matrix = similarity_matrix - min_similarity  # Shift to make all values non-negative
    
# Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

np.fill_diagonal(distance_matrix, 0)

# Flatten the distance matrix to a condensed form (required by scipy)
condensed_distance_matrix = squareform(distance_matrix)

-0.58


In [48]:
print(min_similarity)

-0.58


In [23]:
import statistics

import matplotlib.pyplot as plt



for i in range(1,3000,200):
    n_clusters = 2000 # Adjust this based on your needs
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        affinity="precomputed",  # Use precomputed distances
        linkage="average"  # You can change this to "single", "complete", or "ward"
    )
    
    # Fit the model
    clustering.fit(distance_matrix)
    
    # Get the cluster labels
    labels = clustering.labels_
    
    # Organize genes into clusters
    clusters = {}
    for gene, label in zip(genes, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(gene)
    
    # Print the clusters
    similarity_list = []
    cluster_size_list = []
    for cluster_id, genes_in_cluster in clusters.items():
        similarity_list.append(average_similarity(genes_in_cluster, similarity_matrix, genes))
        cluster_size_list.append(len(genes_in_cluster))
        if "cycle" in genes_in_cluster:
            print(f"Cluster {cluster_id}: {genes_in_cluster}, {average_similarity(genes_in_cluster, similarity_matrix, genes)}")
    
    print(i,statistics.mean(similarity_list),statistics.median(similarity_list))
    # plt.hist(similarity_list, bins=30, color='skyblue', edgecolor='black')
    # plt.show()
    print(i,statistics.mean(cluster_size_list),statistics.median(cluster_size_list))
    
    # plt.hist(cluster_size_list, bins=300, color='skyblue', edgecolor='black')
    # plt.xlim(0, 10)
    # plt.show()
    break

/home/saurav/.local/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


Cluster 1505: ['LHX8', 'HLF', 'GPSM2', 'cycle'], 0.45166666666666666
1 0.2171747841133037 0.37
1 2.271 2.0


In [21]:
def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the average similarity within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Compute the average similarity (excluding diagonal)
    return np.sum(submatrix) / (len(cluster) * (len(cluster) - 1))


In [36]:
import numpy as np

# Assuming 'matrix' is your original NumPy array with NaNs
matrix = np.array([[1, 2, np.nan], [4, np.nan, 6], [7, 8, 9]])

# Remove NaN values while preserving the structure for clustering
mask = ~np.isnan(matrix)
cleaned_matrix = matrix[mask]

print("Original Matrix:")
print(matrix)
print("\nCleaned Matrix (NaNs removed):")
print(cleaned_matrix)

Original Matrix:
[[ 1.  2. nan]
 [ 4. nan  6.]
 [ 7.  8.  9.]]

Cleaned Matrix (NaNs removed):
[1. 2. 4. 6. 7. 8. 9.]


In [37]:
nan_indices = np.argwhere(np.isnan(similarity_matrix))

In [38]:
print(nan_indices[0])

[  1 272]
